In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install transformers
!pip install demoji
!pip install googletrans==3.1.0a0


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip in

In [2]:
import re
from sklearn import metrics
from sklearn.metrics import classification_report

# BERT imports
from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import BertConfig, get_linear_schedule_with_warmup
import torch
# from urllib3.util.ssl_ import ssl
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
print(pd.__version__)
import io
import numpy as np
import unicodedata
from googletrans import Translator
import demoji
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import defaultdict
from sklearn.model_selection import train_test_split
RANDOM_SEED = 7
MAX_LEN = 88
BATCH_SIZE = 80
EPOCHS = 4
N_CLASSES = 2
# PRE_TRAINED_MODEL = "bert-base-uncased"
PRE_TRAINED_MODEL = "bert-base-cased"

1.2.4


## Define Preprocess Operations

## Load Twitter Dataset Files:  Training & Test 

## Downselect the Test data

In [3]:
twitter_train=pd.read_csv('./lower_train.csv',encoding="utf-8",header=None)
twitter_test=pd.read_csv('./lower_test.csv',encoding="utf-8",header=None)
twitter_train=twitter_train[[0,1]]
twitter_train.columns=['tweet','sentiment', ]
twitter_test=twitter_test[[0,1]]
twitter_test.columns=['tweet','sentiment',]

In [4]:
#  Turn the dataset to Binary Classification
twitter_train['sentiment'].replace({4: 1},inplace=True)
twitter_test['sentiment'].replace({4: 1},inplace=True)
twitter_train['sentiment'].replace({0: 0},inplace=True)
twitter_test['sentiment'].replace({0: 0},inplace=True)

# def converter(datainput):
#     t=datainput
#     return eval(t)

# datasets=[twitter_train,twitter_test]
# for i in datasets:
#     i['sentiment']=i['sentiment'].apply(converter)

In [5]:
# Split off the validation set
df_val, df_test = train_test_split(twitter_test, test_size=0.5, random_state=RANDOM_SEED)

In [6]:
# Ensure Binary Classification is set
print(twitter_train.head())
print(twitter_test.head())
print(twitter_train.tail())
print(twitter_test.tail())

                                               tweet  sentiment
0  awww, thats a bummer.  you shoulda got david c...          0
1  is upset that he cant update his facebook by t...          0
2  i dived many times for the ball. managed to sa...          0
3     my whole body feels itchy and like its on fire          0
4  no, its not behaving at all. im mad. why am i ...          0
                                               tweet  sentiment
0  i loooooooovvvvvveee my kindle2. not that the ...          1
1  reading my kindle2...  love it... lee childs i...          1
2  ok, first assesment of the  ...it fucking rock...          1
3  youll love your kindle2. ive had mine for a fe...          1
4  fair enough. but i have the kindle2 and i thin...          1
                                                     tweet  sentiment
1599995  just woke up. having no school is the best fee...          1
1599996  thewdb.com  very cool to hear old walt intervi...          1
1599997  are you ready

In [7]:
twitter_train.shape, twitter_test.shape, df_test.shape, df_val.shape

((1600000, 2), (359, 2), (180, 2), (179, 2))

In [8]:
twitter_train['tweet'][400]

'woke up and was having an accident  &quot;its pushing, its pushing!&quot; he was crying because he couldnt stop from wetting his pants.'

In [9]:
print(twitter_test['tweet'][350])
len(twitter_test[(twitter_test['sentiment']==2)])

getting ready to test out some burger receipes this weekend. bobby flay has some great receipes to try. thanks bobby.


0

In [10]:
# verify GPU availability


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

# Create the model's TOKENIZER

In [11]:

tweet_tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL)


In [12]:
sample_txt = twitter_train['tweet'][400]
tokens = tweet_tokenizer.tokenize(sample_txt)
token_ids = tweet_tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}\n')

print(tweet_tokenizer.unk_token, tweet_tokenizer.unk_token_id)

 Sentence: woke up and was having an accident  &quot;its pushing, its pushing!&quot; he was crying because he couldnt stop from wetting his pants.
   Tokens: ['woke', 'up', 'and', 'was', 'having', 'an', 'accident', '&', 'quo', '##t', ';', 'its', 'pushing', ',', 'its', 'pushing', '!', '&', 'quo', '##t', ';', 'he', 'was', 'crying', 'because', 'he', 'couldn', '##t', 'stop', 'from', 'wet', '##ting', 'his', 'pants', '.']
Token IDs: [8271, 2039, 1998, 2001, 2383, 2019, 4926, 1004, 22035, 2102, 1025, 2049, 6183, 1010, 2049, 6183, 999, 1004, 22035, 2102, 1025, 2002, 2001, 6933, 2138, 2002, 2481, 2102, 2644, 2013, 4954, 3436, 2010, 6471, 1012]

[UNK] 100


In [13]:
encoding = tweet_tokenizer.encode_plus(
                sample_txt,
                max_length=MAX_LEN,
                add_special_tokens=True,      # Add '[CLS]' and '[SEP]'                
                padding="max_length",
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=False,
                return_tensors = 'pt')   # Return PyTorch tensors

encoding.keys()
encoding.to(device)

{'input_ids': tensor([[  101,  8271,  2039,  1998,  2001,  2383,  2019,  4926,  1004, 22035,
          2102,  1025,  2049,  6183,  1010,  2049,  6183,   999,  1004, 22035,
          2102,  1025,  2002,  2001,  6933,  2138,  2002,  2481,  2102,  2644,
          2013,  4954,  3436,  2010,  6471,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]], device='cuda:0')}

In [14]:
print(len(encoding['input_ids'][0]))
print(len(encoding['attention_mask'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])
print(tweet_tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))

55
55
tensor([  101,  8271,  2039,  1998,  2001,  2383,  2019,  4926,  1004, 22035,
         2102,  1025,  2049,  6183,  1010,  2049,  6183,   999,  1004, 22035,
         2102,  1025,  2002,  2001,  6933,  2138,  2002,  2481,  2102,  2644,
         2013,  4954,  3436,  2010,  6471,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], device='cuda:0')
['[CLS]', 'woke', 'up', 'and', 'was', 'having', 'an', 'accident', '&', 'quo', '##t', ';', 'its', 'pushing', ',', 'its', 'pushing', '!', '&', 'quo', '##t', ';', 'he', 'was', 'crying', 'because', 'he', 'couldn', '##t', 'stop', 'from', 'wet', '##ting', 'his', 'pants', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[

# Determine the max length of a tweet sequence of tokens

In [15]:
#  CODE TO DETERMINE MAX SENTENCE LENGTH  Uncommet to compute

# max_len = 0
# token_lens = []

# # For every sentence...
# for sent in twitter_train["tweet"]:

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     tokens = tokenizer.encode(sent, add_special_tokens=True)

#     # Keep track of the length of the tokens
#     token_lens.append(len(tokens))

#     # Update the maximum sentence length.
#     max_len = max(max_len, len(tokens))

# print('Max sentence length: ', max_len)

# sns.displot(token_lens)

# Max sentence length:  230

In [16]:
# sns.distplot(token_lens)
# max(token_lens)

## Create two different approaches to loading data

In [17]:
class TwitterData(Dataset):                                     # TwitterData returns Dictionary and tweet
    def __init__(self, tweet, sentiment, tokenizer, max_len):
        self.tweet = tweet
        self.sentiment = sentiment
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        tweet = str(self.tweet[item])
        sentiment = self.sentiment[item]
        
        encoding = tweet_tokenizer.encode_plus(
                        tweet,
                        max_length=self.max_len,
                        add_special_tokens=True,                      
                        padding="max_length",
                        truncation=True,
                        return_attention_mask=True,
                        return_token_type_ids=False,
                        return_tensors = 'pt')
        return  {  # 'tweet_txt': tweet,
                 'input_ids':encoding['input_ids'].flatten(),
                 'attention_mask':encoding['attention_mask'].flatten(),
        'sentiment': torch.tensor(sentiment, dtype=torch.long)}
        
def create_data_loader(df, tokenizer, max_len, batch_size):  #  TwitterData has dictionary returned
    ds = TwitterData(
        tweet = df.tweet.to_numpy(),
        sentiment = df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len = max_len
    )
    return DataLoader(ds,
                     batch_size = batch_size,
                     num_workers=4,pin_memory=True
                     )

In [18]:
#  CLASS TO READ one row and a Batch
              
class TwitterDataset(Dataset):                        # TwitterDataset tests for memory speeed  returns tensors
    def __init__(self, tweet, sentiment, tokenizer, max_len):
        self.tweet = tweet
        self.sentiment = sentiment
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        tweet = str(self.tweet[item])
        sentiment = self.sentiment[item]
        
        encoding = tweet_tokenizer.encode_plus(
                        tweet,
                        max_length=self.max_len,
                        add_special_tokens=True,                      
                        padding="max_length",
                        truncation=True,
                        return_attention_mask=True,
                        return_token_type_ids=False,
                        return_tensors = 'pt')
        return  torch.stack([encoding['input_ids'],
                 encoding['attention_mask']]), torch.tensor(self.sentiment[item], dtype=torch.long)
    
    
class TwitterBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.snt = torch.stack(transposed_data[0], 0)
        print(f' transposed data {transposed_data}')

    # custom memory pinning method on custom type
    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.snt = self.snt.pin_memory()
        return self

def collate_wrapper(batch):
    return TwitterBatch(batch)


In [19]:
twitter_train.iloc[[0,1],[0]]

,tweet
0,"awww, thats a bummer. you shoulda got david c..."
1,is upset that he cant update his facebook by t...


In [20]:
# for idx,  sent in twitter_train.iloc[[0,1,3],[0]].T.iteritems().to_string():
for idx,  sent in twitter_train.iloc[[0,1,3],[0]].T.iterrows():
    print(sent)


0    awww, thats a bummer.  you shoulda got david c...
1    is upset that he cant update his facebook by t...
3       my whole body feels itchy and like its on fire
Name: tweet, dtype: object


## Build the four data loaders:  train_data_loader and training_set_loader

In [21]:
# CODE TO READ the DATA SETS  - two for dict and two for tensor
train_data_loader = create_data_loader(twitter_train, tweet_tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tweet_tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tweet_tokenizer, MAX_LEN, BATCH_SIZE)
#-------------------------------------------------------------------------------------------
# Create speed test datasets using the custom classes
train_dataset = TwitterDataset(twitter_train['tweet'].to_numpy(),twitter_train['sentiment'].to_numpy(),tweet_tokenizer, MAX_LEN)
test_dataset = TwitterDataset(twitter_test['tweet'].to_numpy(),twitter_test['sentiment'].to_numpy(),tweet_tokenizer, MAX_LEN)

# Create speet test data loaders for our datasets; shuffle for training, not for validation
training_set_loader = DataLoader(train_dataset,
                               shuffle=True,
                               num_workers=4, collate_fn=collate_wrapper,
                               batch_size=BATCH_SIZE,drop_last=True,pin_memory=True)
validation_set_loader = DataLoader(test_dataset, 
                               shuffle=False,
                               num_workers=4, collate_fn=collate_wrapper,
                               batch_size=BATCH_SIZE,drop_last=True,pin_memory=True)
# pin_memory=True,collate_fn=collate_wrapper,sampler = SequentialSampler(test_dataset), drop_last=True
# pin_memory=True,collate_fn=collate_wrapper,sampler = RandomSampler(train_dataset), num_workers=2, drop_last=True, batch_size=4,


print('Training set has {} instances'.format(len(train_dataset)))
print('Validation set has {} instances'.format(len(test_dataset)))

print(f' data shapes: {twitter_train.shape, twitter_test.shape, df_test.shape, df_val.shape}{train_dataset}{test_dataset}')


Training set has 1600000 instances
Validation set has 359 instances
 data shapes: ((1600000, 2), (359, 2), (180, 2), (179, 2))<__main__.TwitterDataset object at 0x2b751e19b1f0><__main__.TwitterDataset object at 0x2b751e19b4f0>


In [22]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['input_ids', 'attention_mask', 'sentiment'])

In [23]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['sentiment'].shape)

torch.Size([80, 55])
torch.Size([80, 55])
torch.Size([80])


In [24]:
# inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
# tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
# dataset = TensorDataset(inps, tgts)


# loader = DataLoader(dataset, batch_size=2, collate_fn=collate_wrapper,
#                     pin_memory=True)

# for batch_ndx, sample in enumerate(loader):
#     print(sample.inp.is_pinned())
#     print(sample.tgt.is_pinned())


for batch_ndx, sample in enumerate(training_set_loader):
    print(sample.inp.is_pinned())
    print(sample.snt.is_pinned())
    break

 transposed data [(tensor([[[  101,  2061,  3565,  6517,  3892,  1012,  2026,  2767, 18193,  2015,
           3566,  2979,  2185,  2126,  2205,  2574,  1010,  2016,  2001,  2066,
           1037,  3566,  2000,  2033,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0]],

        [[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0]]]), tensor([[[  101,  4067,  2017,   999,  1998,  1010, 18411,  2860,  1010,  2017,
    

In [25]:
for input1 , target in train_dataset:
    print(input1)
    break
    
for input1, target in test_dataset:
    print(input1)
    break
    
for input1, target in test_dataset:
    print(input1)
    break
    
    
data = next(iter(train_data_loader))
print(data)
data2 = next(iter(training_set_loader))
print(data.keys())
print(data2)

# dict_keys(['tweet_txt', 'input_ids', 'attention_mask', 'sentiment'])
print(data['input_ids'].shape,data['attention_mask'].shape,data['sentiment'].shape)

tensor([[[  101, 22091,  2860,  2860,  1010,  2008,  2015,  1037, 26352,  5017,
           1012,  2017,  2323,  2050,  2288,  2585, 12385,  1997,  2353,  2154,
           2000,  2079,  2009,  1012,  1025,  1040,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0]],

        [[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0]]])
tensor([[[  101,  1045,  8840,  9541,  9541,  9541,  4492,  2615,  2615,  2615,
           2615,  3726, 

In [26]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([80, 55])
torch.Size([80, 55])


# DEFINE THE BERT MODEL

In [27]:
bert_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL,  return_dict=False)   # return_dict=True
bert_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [28]:
last_hidden_state, pooler_output= bert_model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'],
)

In [29]:
print(last_hidden_state.shape)
print(pooler_output.shape)


torch.Size([1, 55, 768])
torch.Size([1, 768])


In [30]:
bert_model.config.hidden_size

768

In [31]:
# Get all of the model's parameters as a list of tuples.
params = list(bert_model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (

In [32]:
# class Classifier(torch.nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()
#         self.base_model=AutoModel.from_pretrained(PRE_TRAINED_MODEL,return_dict=False)
#         self.fc = nn.Linear(768, 1)

#     def forward(self, input_ids, attention_mask):
#         bert_assignment, bert_output = self.base_model(encoding['input_ids'],  encoding['attention_mask'])
#         return self.fc(bert_output)

In [33]:
# model = Classifier()
# model.to(device)

In [34]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL, return_dict=False)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [35]:
class_names=['positive','negative']
model = SentimentClassifier(len(class_names))
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
# F.softmax(model(input_ids, attention_mask), dim=1)

model_output = model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'],
)

print(model_output)

tensor([[0.8172, 0.1216]], device='cuda:0', grad_fn=<AddmmBackward>)


## CREATE LOSS FUNCTION for training the network

In [37]:
# loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, eps = 1e-8, correct_bias=False) # args.adam_epsilon  - default is 1e-8.
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [38]:


def train_one_epoch(training_loader, epoch_index, tb_writer):
    running_loss = 0
    losses = []
    last_loss = 0.
    correct_predictions = 0
    n_examples = 0
    

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, d in enumerate(training_loader):
#         print(data, i)
        # Every data instance is an input_id, attention_mask + sentiment group
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        sentiment = d["sentiment"].to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(
              input_ids=input_ids,
              attention_mask=attention_mask
        )

#         print(f' outputs {outputs} {len(outputs[0])}')

        # Translate the prediction to the class outputs -1 and 1
        _, preds = torch.max(outputs, dim=1)

#         print(f'pred {preds}')

        # Compute the loss and its gradients
        loss = loss_fn(outputs.squeeze(1).to(device), sentiment.reshape(-1).to(device))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Adjust learning weights
        optimizer.step()
        scheduler.step()

        # Gather data and report
        correct_predictions += torch.sum(preds == sentiment)
        losses.append(loss.item())
#         print(f'loss {loss}')
        running_loss += loss.item()
        n_examples += BATCH_SIZE

        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.    # Every data instance is an input + label pair



    return last_loss, correct_predictions.double() / n_examples, np.mean(losses)

# Test ONE EPOCH RUN

In [39]:
running_loss = 0.
last_loss = 0.
correct_predictions = 0
n_examples = 0

# Here, we use enumerate(training_loader) instead of
# iter(training_loader) so that we can track the batch
# index and do some intra-epoch reporting
for i, d in enumerate(train_data_loader):
    print(data, i)
    # Every data instance is an input_id, attention_mask + sentiment group
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    sentiment = d["sentiment"].to(device)

    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
    )

    print(f' outputs {outputs} {len(outputs[0])}')
    
    # Translate the prediction to the class outputs -1 and 1
    _, preds = torch.max(outputs, dim=1)

    print(f'pred {preds}')

    # Compute the loss and its gradients
    loss = loss_fn(outputs.squeeze(1).to(device), sentiment.reshape(-1).to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    # Adjust learning weights
    optimizer.step()
    scheduler.step()

    # Gather data and report
    correct_predictions += torch.sum(preds == sentiment)
    running_loss += loss
    print(f'loss {loss.item()}')
    print(f'loss2 {loss}')
    n_examples += BATCH_SIZE

    if i % 1000 == 999:
        last_loss = running_loss / 1000 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        tb_x = epoch_index * len(training_loader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.    # Every data instance is an input + label pair
    break

{'input_ids': tensor([[  101, 22091,  2860,  ...,     0,     0,     0],
        [  101,  2003,  6314,  ...,     0,     0,     0],
        [  101,  1045, 11529,  ...,     0,     0,     0],
        ...,
        [  101,  2053,  2047,  ...,     0,     0,     0],
        [  101,  1045,  2228,  ...,     0,     0,     0],
        [  101, 16278,  2339,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'sentiment': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])} 0
 outputs tensor([[ 0.6469,  0.5203],
        [ 0.1619,  0.8668],
        [ 0.8561,  0.4819],
    

In [49]:
def eval_model(model, data_loader, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  running_vloss = 0.0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      sentiment = d["sentiment"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs.squeeze(1).to(device), sentiment.reshape(-1).to(device))
      correct_predictions += torch.sum(preds == sentiment)
      losses.append(loss)
      running_vloss += loss
  return running_vloss/n_examples, correct_predictions.double() / n_examples, np.mean(losses, dtype=float)

In [ ]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter 
training_stats = []

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/tweet_trainer_{}'.format(timestamp))
epoch_number = 0

history = defaultdict(list)

EPOCHS = 3

best_vloss = 1_000_000.

for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss=0
    avg_loss, train_acc, train_loss = train_one_epoch(train_data_loader, epoch_number, writer)
    
    print(f'Train loss {train_loss} accuracy {train_acc}')
    model_path = './finetunedBert.ckpt'
    torch.save(model.state_dict(), model_path)
    
    # We don't need gradients on to do reporting
    model.train(False)
    avg_tloss=0
    avg_tloss, test_acc, test_loss = eval_model(
        model, 
        test_data_loader,
        len(df_test)
      )
    print(f'Val   loss {test_loss} accuracy {test_acc}')
    print()
#     running_vloss = 0.0
#     for i, vdata in enumerate(test_data_loader):
#         vinputs, vlabels = vdata
#         voutputs = model(vinputs)
#         vloss = loss_fn(voutputs, vlabels)
#         running_vloss += vloss

#     avg_vloss = running_vloss / (i + 1)

    

    avg_vloss=0
    avg_vloss, val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        len(df_val)
      )
    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()
    print('LOSS train {} test {} valid {}\n'.format(avg_loss, avg_tloss, avg_vloss))
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['test_acc'].append(test_acc)
    history['test_loss'].append(test_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
#     running_vloss = 0.0
#     for i, vdata in enumerate(val_data_loader):
#         vinputs, vlabels = vdata
#         voutputs = model(vinputs)
#         vloss = loss_fn(voutputs, vlabels)
#         running_vloss += vloss

#     avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()


    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


Epoch 1/3
----------
EPOCH 1:
  batch 1000 loss: 0.03904700762505763
  batch 2000 loss: 1.4016528937190742e-06
  batch 3000 loss: 9.218301677833551e-07
  batch 4000 loss: 6.565688534010405e-07
  batch 5000 loss: 4.850029256431298e-07
  batch 6000 loss: 3.610878840163423e-07
  batch 7000 loss: 2.681582936503446e-07
  batch 8000 loss: 1.9992737124141512e-07
  batch 9000 loss: 1.488476880098233e-07
  batch 10000 loss: 1.0945497884051747e-07
  batch 11000 loss: 0.0654897872396898
  batch 12000 loss: 1.8778639810079767e-06
  batch 13000 loss: 1.1279635552909894e-06
  batch 14000 loss: 7.819381461899866e-07
  batch 15000 loss: 5.733458935708313e-07
  batch 16000 loss: 4.2639219984152986e-07


Val   loss 6.718822320302327 accuracy 0.5555555555555556
Val   loss 9.071404139200846 accuracy 0.4581005586592179

LOSS train 1.3983099503178664e-07 test 0.1119803786277771 valid 0.1520346850156784
LOSS train 1.3983099503178664e-07 valid 0.1520346850156784


In [ ]:
model_path = './finetunedBert.ckpt'
output_model = './altsaveBert.ckpt'
torch.save(model.state_dict(), model_path)
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      sentiment = d["sentiment"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(sentiment)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
test_data_loader = create_data_loader(twitter_test, tweet_tokenizer, MAX_LEN, BATCH_SIZE)
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)
print(classification_report(y_true=y_test,y_pred=y_pred_class))

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()